In [1]:
from pathlib import Path
import cv2

from core.data.services.default_video_service import DefaultVideoService as VideoService
from core.infra.scene_objects.corners import Corners

/Users/yann/Documents/Projects/Python/basket-tracking/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
video_service = VideoService()

In [ ]:
Corners.from_rectangle()

In [3]:
input_video_path = Path().cwd().joinpath("assets").joinpath("videosample_10s.mp4")
input_video_path.is_file()
video_service = VideoService()

scenes = video_service.getScenesFromVideo(video_path=input_video_path, region_of_interest=None)

scene = next(scenes)

/Users/yann/Documents/Projects/Python/basket-tracking/.venv/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /Users/distiller/project/pytorch/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:
frame = scene.draw_frame()

In [5]:
window_name = 'scene'
  
# Using cv2.imshow() method 
# Displaying the image 
cv2.imshow(window_name, frame)
  
#waits for user to press any key 
#(this is necessary to avoid Python kernel form crashing)
cv2.waitKey(0) 
  
#closing all open windows 
cv2.destroyAllWindows() 

In [6]:
from core.data.services.default_image_service import DefaultImageService as ImageService

In [7]:
image_service = ImageService()

In [8]:
image_service.darwRectangleInFrame()

AttributeError: 'DefaultImageService' object has no attribute 'darwRectangleInFrame'

: 

In [ ]:
len(scene.players[0].position[0])

1